
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

# New Section

In [6]:
!pip install scikit-learn==0.19.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 8.1 MB/s 
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Failed to build scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
    Running setup.py install for scikit-learn ... error
  Rolling back uninstall of scikit-learn
  Moving to /usr/local/lib/python3.7/dist-packages/scikit_learn-1.0.2.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~cikit_learn-1.0.2.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/scikit_learn.libs/
   from /usr/local/lib/python3.7/dist-packages/~cikit_learn.libs
  Moving to /usr/local/lib/python3.7/dist-packages/sklearn/
   from /usr/local/lib/python3.7/dist-packages/~klearn
ERROR: Command errored out with exit status 1

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [9]:
ls

Actor_01/  Actor_07/  Actor_13/  Actor_19/  modelForPrediction1.sav
Actor_02/  Actor_08/  Actor_14/  Actor_20/  Recording.wav
Actor_03/  Actor_09/  Actor_15/  Actor_21/
Actor_04/  Actor_10/  Actor_16/  Actor_22/
Actor_05/  Actor_11/  Actor_17/  Actor_23/
Actor_06/  Actor_12/  Actor_18/  Actor_24/


In [10]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [11]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [12]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'No_Depression',
  '02':'No_Depression',
  '03':'No_Depression',
  '04':'D',
  '05':'angry',
  '06':'Depression',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['No_Depression', 'No_Depression','Depression']


In [13]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [14]:
#split the dataset
x_train,x_test,y_train,y_test=load_data()

In [15]:
x_train

array([[-6.67103821e+02,  6.13767700e+01, -9.01065946e-01, ...,
         2.47099379e-05,  1.72176005e-05,  5.94455378e-06],
       [-4.09896820e+02,  1.51403370e+01, -2.24304562e+01, ...,
         4.35975147e-03,  1.97707559e-03,  1.23020017e-03],
       [-7.15719116e+02,  4.05106316e+01, -4.56917381e+00, ...,
         1.34494312e-05,  8.77831280e-06,  8.59934789e-06],
       ...,
       [-6.35971558e+02,  3.48089714e+01, -1.56437559e+01, ...,
         4.53210559e-05,  4.12219451e-05,  3.12678312e-05],
       [-7.23201050e+02,  3.54729919e+01,  3.19336295e+00, ...,
         2.77742893e-05,  1.95146604e-05,  6.86829981e-06],
       [-7.25144592e+02,  6.24018250e+01,  2.32453048e-01, ...,
         4.45031105e-07,  2.49656267e-07,  1.16060498e-07]])

In [16]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(544, 137)


In [17]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [18]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [19]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [20]:
#Predict for the test set
y_pred=model.predict(x_test)

In [21]:
y_pred

array(['No_Depression', 'No_Depression', 'No_Depression', 'Depression',
       'Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'Depression', 'No_Depression', 'No_Depression', 'Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'Depression', 'No_Depression', 'Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'Depression', 'Depression',
       'Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 'No_Depression',
       'No_Depression', 'No_Depression', 'No_Depression', 

In [22]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 86.86%


In [23]:
from sklearn.metrics import accuracy_score, f1_score

In [24]:
f1_score(y_test, y_pred,average=None)

array([0.68965517, 0.91666667])

In [25]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,No_Depression,No_Depression
1,No_Depression,No_Depression
2,No_Depression,No_Depression
3,Depression,Depression
4,No_Depression,Depression
5,No_Depression,No_Depression
6,No_Depression,No_Depression
7,No_Depression,No_Depression
8,Depression,Depression
9,No_Depression,No_Depression


In [26]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [29]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab Notebooks/speech-emotion-recognition-ravdess-data/Recording_1.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['No_Depression'], dtype='<U13')